In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# from __future__ import division
import numpy as np
# import scipy.signal as sg
# from scipy.fftpack import rfft, fftfreq
from scipy.io import wavfile
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches
# %matplotlib inline
# import matplotlib.colors as colors
# from pydub import AudioSegment
import math as m
import tensorflow as tf
import utilities as util

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [77]:
def gather_samples():
    '''
    Gather all of the files used to train the model
    Trombone and Guitar to setup a binary network
    '''
    sample_files = []
    location = '../samples/'
    instruments = ['Trombone', 'Guitar']
    for name in instruments:
        for note in (util.note_names('C1', 'C2')):
            for j in range(1, 4):
                file_name = location +name+"/"+name +"_"+note[:-1]+"("+str(j)+").wav"
                sample_files.append(file_name)
    return sample_files

In [12]:
nps_uniq, dfreqs, notes = util.precompute_spect()
smallest_nps = 44100/24 * 4/3 
smallest_nps = 2**int(np.log2(smallest_nps))
note_freq = util.frequencies('A0', 'D8')
sample_files = gather_samples()

In [44]:
def tf_normal_peak(vals, dfreq):
    '''
    Calculate the normal peak of the frequencies with tensors
    '''
    #print('vals correct')
    sums = tf.reduce_sum(vals, 0, keepdims=True)
    #print('sums calculated')
    weights = vals/sums
    #print('weights calculated')
    freqs = tf.range(tf.cast(tf.shape(vals)[0], 'float64'), dtype='float64')
    #print('freqs calculated')
    mean = tf.reduce_sum(
        tf.cast(weights, 'float64') * tf.cast(freqs[:,None], 'float64'),  
        axis=0,
        keepdims=True)
    #print('mean calculated')
    var = tf.truediv(tf.reduce_sum(tf.cast(((freqs[:,None]-mean)**2), 'float64')*tf.cast(weights, 'float64'), 0),dfreq)
    #print('var calculated')
    denom_var = tf.multiply(tf.constant(2.0, dtype='float64'), m.pi)
    #print('denominator variable calculated')
    denom_var_mult = tf.scalar_mul(denom_var, var)
    #print('multiplied denominator calculated')
    denom = tf.sqrt(tf.abs(denom_var_mult))
    #print('denominator calculated')
    dst_peak = tf.truediv(tf.constant(1.0, dtype='float64'),denom)
    #print('distribution peak calculated')
    peak = (tf.cast(dst_peak, 'float64') * tf.cast(dfreq, 'float64') * tf.cast(tf.squeeze(sums), 'float64')) if (dst_peak != 'Nan') else 0
    #print('peak calculated')
    return peak

In [15]:
def preprocess(filename):
    # Gathering the Data
    # Read the Wave File
    # TODO - In terms of Tensorflow?
    fs, wav = wavfile.read(filename)
    wav = wav.astype(np.double)
    # Calculate the Spectrogram
    freq, time, spectrogram = tf_spect(wav, fs)
#     arg_sort = np.argsort(spectrogram,  axis=0)
    return spectrogram

In [45]:
def tf_spect(wav, fs):   
    data = []
    # Caclculating the Spectrogram
    for nperseg in nps_uniq:
#         print('...')
        signals = tf.placeholder(tf.float32)

#         frames = tf.contrib.signal.frame(signals, frame_length=np.int32(nperseg),
#                                          frame_step=np.int32(nperseg-smallest_nps))
        stfts = tf.contrib.signal.stft(signals, frame_length=np.int32(nperseg),
                                       frame_step=np.int32(nperseg-smallest_nps),
                                       fft_length=np.int32(nperseg))
        #   power_spectrograms = tf.real(stfts * tf.conj(stfts))
        magnitude_spectrograms = tf.abs(stfts)
        data.append(magnitude_spectrograms)
    # Empty new spectrogram
    # TODO: CALCULATE data[0][1]
    # This will help to define the shape ::
    full_spec = tf.Variable(tf.zeros((len(note_freq)-1, tf.size(data[0][1])), tf.float64), validate_shape=False)
    
    # Calculate the normal peaks in order to make a more clear spectrogram
    for i, (nps_ind, slc) in enumerate(notes):
        # slc = (start, end)
#         print(tf.shape(full_spec))
        tf_spec = full_spec[nps_ind]
#        print('-')
        new_vars = tf_spec#(tf.slice(tf_spec, [slc[0]], [slc[1]-slc[0]]))
        if vals.shape[0] == 1: return vals

        tf_np = tf_normal_peak(new_vars, dfreqs[nps_ind])
#        print('--')
        time = tf.shape(tf_spec)[0]
        full_spec[i,:tf.size(time)].assign(tf_np)
        #full_spec = tf.Variable(full_spec[i,:tf.size(time)].assign(tf_np))
        #full_spec = tf.Variable(full_spec[i,:tf.size(time)].assign(tf_np), validate_shape=False)
        # Sliced assignment is only supported for variables
    
    # Return the note frequencies, original data, and spectrogram with normalized peaks
    return full_spec

In [46]:
f, t, s = preprocess('../samples/Trombone/Trombone_A#(1).wav')
# Sliced assignment is only supported for variables

In [78]:
# Preprocess all of the sample files
# Create lists of all of the frequencies, times and spectrograms
all_freqs = []
all_time = []
all_specs = []
for sf in sample_files:
    s = preprocess(sf)
#     all_freqs.append(f)
#     all_time.append(t)
    all_specs.append(s)

/Users/Peeke/Honors/.env/lib/python3.6/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [79]:
# Create a Network (1)
# x = tf.placeholder(shape=[None, 2], dtype=tf.float32)
x = tf.placeholder(shape=[None, 2], dtype=tf.float32)
nn = tf.layers.dense(x, 3)#, activation=tf.nn.sigmoid)
nn = tf.layers.dense(nn, 5)#, activation=tf.nn.sigmoid)
encoded = tf.layers.dense(nn, 2)#, activation=tf.nn.sigmoid)
ret = tf.layers.flatten(encoded)

In [80]:
with tf.Session() as sess:
#     for spectro in all_specs:
    for step in range(10000):
        sess.run(ret, feed_dict={x:tf.convert_to_tensor(all_specs, dtype=tf.float64)})
        # sess.run(ret, feed_dict={x:tf.Variable(all_specs, dtype=tf.float64)})
        # tf.convert_to_tensor(all_specs, np.ndarray)
        # sess.run(ret, feed_dict={x:tf.convert_to_tensor(all_specs, dtype=tf.float64)})
        if step % 5 == 0:
            print("step: {}, value: {}".format(step, val))


TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles.For reference, the tensor object was Tensor("packed_1:0", dtype=float64) which was passed to the feed with key Tensor("Placeholder_1148:0", shape=(?, 2), dtype=float32).

In [34]:
# Create a Convolutional Network (2)
x = tf.placeholder(shape=[None, None], dtype=tf.float32)
nn = tf.layers.conv2d(x, 3, 3)#, activation=tf.nn.sigmoid)
encoded2 = tf.layers.flatten(nn, 2)#, activation=tf.nn.sigmoid)

ValueError: Input 0 of layer conv2d_8 is incompatible with the layer: expected ndim=4, found ndim=2. Full shape received: [None, None]

In [27]:
import keras

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Generate dummy data
import numpy as np
data = np.random.random((1000, 100))
labels = np.random.randint(2, size=(1000, 1))

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, labels, epochs=10, batch_size=32)


ModuleNotFoundError: No module named 'keras'